In [1]:
# INIT PYTHON IMPORT
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 드라이버 패스는 자신의 위치에 맞게 수정
driver_path = 'chromedriver.exe'

# 웹크롤링
* 사용페이지 : op.gg (게임 리그 오브 레전드의 전적을 보여주는 웹사이트)
* 유저의 최근 전적에 대한 데이터 프레임을 생성
* 생성할 항목(columns) : 게임 플레이 날짜, 게임 결과, 플레이 챔피언, K(킬수), D(데스), A(어시스트), 게임 길이
* 날짜는 년월일 시간이 출력되지만, 나는 월일만 사용할 것이므로 split과 replace를 이용한다

In [4]:
url = 'https://op.gg/'
driver = webdriver.Chrome(driver_path)
driver.get(url)

#테스트를 위해 닉네임은 아무거나
target = '365일간의'
elem = driver.find_element_by_name("userName")
elem.clear()
# 검색 구현
elem.send_keys(target)
elem.send_keys(Keys.RETURN)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "GameMoreButton"))
)

#솔로랭크 전적이 부족해 오류가 발생할 것을 대비해 자유랭크 전적을 탐색할 예정 / 탐색할 정보의 개수 : 40
#op.gg의 경우 전적을 모든 게임으로 처음에 보여주기 때문에, 자유랭크로 옴겨야함
path = '#right_gametype_flexranked > a'
driver.find_element_by_css_selector(path).click()
time.sleep(4)

#자유랭크 탭으로 옮겼지만, opgg는 기본적으로는 페이지당 20개의 항목만을 지원하기 때문에, 더보기를 눌러 보여주는 전적의 수를 40개로 확장을 해야함
path2 = '#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div.GameMoreButton.Box'
driver.find_element_by_css_selector(path2).click()
time.sleep(2)

#pagesource를 가지고 오기
html = driver.page_source
time.sleep(1)
driver.close()
#print(html)

In [5]:
# Beautiful Soup으로 데이터 분류하기
soup = BeautifulSoup(html)
date = []
result = []
champion = []
kill = []
death = []
assist = []
game_length = []

# 데이터 처리하기

# 게임 결과 구하기
d1 = soup.select('#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div.GameItemList div.GameResult')
for item in d1:
 #   date.append(item.select('.TimeStamp span').get_text())
    result.append(item.get_text().replace('\n', '').replace('\t', ''))

# 게임플레이 날짜 구하기
d2 = soup.select('#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content div.GameStats > div.TimeStamp > span')
for item in d2:
    text = item['title'][6:].split(' 오')[0]
    
    date.append(text)

# 플레이 챔피언 구학
d3 = soup.select('#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content div.GameSettingInfo > div.ChampionName > a')
for item in d3:
    champion.append(item.get_text())

# 게임 길이 구하기
d4 = soup.select('#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content div.GameStats > div.GameLength')
for item in d4:
    game_length.append(item.get_text())

# 스코어(KDA) 구하기
d5 = soup.select('#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div.GameItemList div > div.Content > div.KDA > div.KDA span')
kda_d = []
for item in d5:
    kda_d.append(item.get_text())
for i in range(len(kda_d)):
    if(i % 3 == 0):
        kill.append(kda_d[i])
    elif(i % 3 == 1):
        death.append(kda_d[i])
    elif(i % 3 == 2):
        assist.append(kda_d[i])

In [11]:
#데이터 프레임 생성하기
data = []
for i in range(40):
    try:
        data.append([date[i], result[i], champion[i], kill[i], death[i], assist[i], game_length[i]])
    except IndexError:
        pass
#print(data)
columns = ['date', 'result', 'champion', 'kill',' death', 'assist', 'game_length']
df = pd.DataFrame(data=data, columns = columns)

In [12]:
#데이터 프레임 출력하기
print('{0} 님의 전적'.format(target))
print(df)

365일간의 님의 전적
      date result champion kill  death assist game_length
0   9월 11일     승리    트린다미어    1      3      4     20분 31초
1    9월 5일     패배     모르가나    0     11     19      45분 6초
2    9월 5일     패배       자야    2     12      6     32분 20초
3    9월 5일     패배       잭스    5     11      6     31분 13초
4    9월 5일     승리       아리    4      4     13     26분 41초
5    9월 3일     승리    하이머딩거   11      7      9      31분 5초
6    9월 3일     승리     다이애나   10     10     11     32분 53초
7    9월 2일     패배     알리스타    2     15      9     30분 24초
8    9월 2일     승리     이렐리아    5     12      5     35분 23초
9    9월 2일     패배       베인    3     11      5     22분 39초
10   9월 2일     승리     킨드레드    3      1      8     24분 35초
11   9월 2일     승리    마스터 이    6      7      6      29분 3초
12   9월 2일     승리     킨드레드    6      0      1     16분 46초
13   9월 2일     승리      리 신   10      3     17      33분 9초
14  8월 31일     승리       아리   10      1      7      28분 1초
15  8월 28일     승리    하이머딩거    1      3     10     20분 22초
1